In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/kaggle/input/titanic/train.csv')
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
extra = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df.drop(['Name','Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df = pd.merge(df, pd.get_dummies(df, columns=['Sex']))
df['Target'] = df['Survived']
df.drop('Survived', axis=1, inplace=True)
df['Age'] = df['Age'].fillna(np.mean(df['Age']))
df.drop('Sex', axis=1,inplace=True)
X = df.iloc[:,:8]
y = df.iloc[:,8]
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
dftest.drop(['Name','Ticket','Cabin','Embarked'], axis=1, inplace=True)
dftest = pd.merge(dftest, pd.get_dummies(dftest, columns=['Sex']))
dftest['Age'] = dftest['Age'].fillna(np.mean(dftest['Age']))
dftest.drop('Sex', axis=1,inplace=True)
dftest['Fare'] = dftest['Fare'].fillna(np.mean(dftest['Fare']))


In [3]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.1, max_depth=3).fit(X_train, y_train)

clf.score(X_test, y_test)

0.8161434977578476

In [4]:
from sklearn.model_selection import GridSearchCV

clf = GradientBoostingClassifier(learning_rate=0.01, max_depth=4, n_estimators=200)

clf.fit(X, y)


GradientBoostingClassifier(learning_rate=0.01, max_depth=4, n_estimators=200)

In [5]:
results = pd.DataFrame({'PassengerId':dftest['PassengerId'], 'Survived':clf.predict(dftest)})

results

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [6]:
results.to_csv('submission.csv', index=False)

In [7]:
gbt_clf = clf

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=10,max_iter=1000).fit(X_train, y_train)
clf.score(X_test, y_test)

clf = LogisticRegression(C=10,max_iter=1000).fit(X, y)

log_clf = clf

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train)
svm_clf.score(X_test, y_test)

0.6636771300448431

In [10]:
svm_clf = SVC()
grid_values = {'kernel': ['poly', 'linear', 'rbf', 'sigmoid']}
grid_clf_acc = GridSearchCV(svm_clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)

grid_clf_acc.best_estimator_


SVC(kernel='linear')

In [11]:
grid_clf_acc.score(X_test, y_test)

0.7802690582959642

In [12]:
svm_clf = SVC(kernel='linear').fit(X,y)

In [13]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

rfc_clf = RandomForestClassifier(max_depth=8, n_estimators=64).fit(X,y)

In [14]:
voting_clf = VotingClassifier(estimators = [('rfc', rfc_clf),('lg', log_clf), ('gbt', gbt_clf), ('svc', svm_clf)], voting="hard", n_jobs=-1)
voting_clf.fit(X,y)

VotingClassifier(estimators=[('rfc',
                              RandomForestClassifier(max_depth=8,
                                                     n_estimators=64)),
                             ('lg', LogisticRegression(C=10, max_iter=1000)),
                             ('gbt',
                              GradientBoostingClassifier(learning_rate=0.01,
                                                         max_depth=4,
                                                         n_estimators=200)),
                             ('svc', SVC(kernel='linear'))],
                 n_jobs=-1)

In [15]:
resultz = pd.DataFrame({'PassengerId':dftest['PassengerId'], 'Survived':voting_clf.predict(dftest)})

resultz

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [16]:
resultz.to_csv('submission.csv',index=False)